# 10. Gépi tanulás 1. – HunTag3

>Számítógépes nyelvészet, 2018 tavasz

>Simon Eszter, Mittelholcz Iván

>MTA, Nyelvtudományi Intézet

## Előkészületek

* [HunTag3](https://github.com/ppke-nlpg/HunTag3) letöltése
* kitömörítése

In [8]:
%%bash

wget -q -c https://github.com/ppke-nlpg/HunTag3/archive/master.zip
unzip -q -u master.zip

## Adatok

Forrás: http://hlt.sztaki.hu/resources/hunnerwiki.html

Oszlopok:

1. **token**
2. egyértelműsített morfológiai elemzés (KR kód)
3. lemma
4. **gold standard**

In [102]:
!head -n 13 huwiki.ner.sample.train.tsv

Legkorábbi	ADJ	korai	O
festményei	NOUN<PLUR><POSS>	festmény	O
Grazban	NOUN<CAS<INE>>	graz	B-LOC
(	PUNCT	(	O
1742-ből	NOUN<CAS<ELA>>	1742	O
)	PUNCT	)	O
és	CONJ	és	O
Szlovéniában	NOUN<CAS<INE>>	szlovénia	B-LOC
(	PUNCT	(	O
1744-45-ből	NOUN<CAS<ELA>>	1744-45	O
)	PUNCT	)	O
találhatók	ADJ<PLUR>	található	O
.	PUNCT	.	O


## Features

A *feature*-öket *on the fly* generáljuk a tanítás során rövid függvényekkel, melyek bemenetüket a TSV valamelyik oszlopából vehetik.

Ezen függvények helye a *features.py* fájlban van. A HunTag3-mal jön egy alap *features.py*, de sajátot is írhatunk.

Hogy milyen featur-öket akarunk használni, és melyik feature a TSV melyik oszlopát használja, azt az általunk írt konfigfájlban tudjuk megadni. A konfigurációs beállításokat [YAML](https://en.wikipedia.org/wiki/YAML) formátumban kell megadni.

* *name*: a *most-informative-features* kimenetében ezen a néven jelenik meg a *feature*
* *type*: lehet token, sentence, vagy lexical, ezek a feature-ök függvényeinek is prefixei egyben
* *actionName*: a függvény neve, *type* prefix nélkül
* *fields*: TSV oszlop száma (0-ával kezdődik)

**konfig**

In [136]:
%%writefile HunTag3-master/myconf.yaml

%YAML 1.1
---
default:
  cutoff: 1  #  1 if not set
  radius: 3  # -1 if not set

features:
 - # szóalak
    name: token
    type: token
    actionName: getToken
    fields: 0
 - # morf elemzés
    name: ana
    type: token
    actionName: getAna
    fields: 1
 - # csupa nagybetű-e
    name: isUpper
    type: token
    actionName: isUpper
    fields: 0
...



Overwriting HunTag3-master/myconf.yaml


**features**

In [128]:
%%writefile HunTag3-master/features.py

def token_isUpper(token, _=None):
    return [int(token.isupper())]

def token_getToken(token, _=None):
    return [token]

def token_getAna(ana, _=None):
    return [ana]


Overwriting HunTag3-master/features.py


## Tanítás

### megfigyelési modell

In [130]:
%%bash

cd HunTag3-master/
python3 huntag.py train -i ../huwiki.ner.sample.train.tsv -c myconf.yaml -m NER
ls -l NER.model

-rw-r--r-- 1 mittelholcz mittelholcz 1577355 ápr   24 17:02 NER.model


featurizing sentences...1000...1000...done!
creating training problem...done!
training with option(s) "None"...done
saving model...done
saving feature and label lists...done


### átmenet modell

In [131]:
%%bash

cd HunTag3-master/
python3 huntag.py transmodel-train -i ../huwiki.ner.sample.train.tsv -m NER

lambda1: 0.07944765574823379
lambda2: 0.7835260115606937
lambda3: 0.1370263326910726


## Címkézés, tesztelés, kiértékelés

### címkézés

In [132]:
%%bash

cd HunTag3-master/
python3 huntag.py tag -i ../huwiki.ner.sample.test.tsv -c myconf.yaml -m NER -o huwiki.ner.test.tagged.tsv

loading transition model...done
loading observation model...done
500...done


### kiértékelés

In [133]:
%%bash

cd HunTag3-master/
python3 utils/eval.py -c -i huwiki.ner.test.tagged.tsv

processed 7232 tokens with 131 phrases;found: 17 phrases; correct: 8.
accuracy:  96.96%; precision:  47.06%;recall:   6.11%; FB1:  10.81
               LOC:precision:   0.00%;recall:   0.00%; FB1:   0.00	0
              MISC:precision:   0.00%;recall:   0.00%; FB1:   0.00	0
               ORG:precision:   0.00%;recall:   0.00%; FB1:   0.00	0
               PER:precision:  47.06%;recall:  26.67%; FB1:  34.04	17


In [135]:
%%bash

cd HunTag3-master/
python3 huntag.py most-informative-features -m NER -c myconf.yaml -i ../huwiki.ner.sample.test.tsv >NER.mostinf

featurizing sentences...500...done!
creating training problem...done!


## Feladatok

Implementáljatok pár új feature-t és értékeljétek ki ezekkel a ner-taggert.
Az új feature-ök legyenek:

* szófaj (része az elemzésnek a 2. oszlopban)
* lemma (3. oszlop)
* nagybetűvel kezdődik-e
* van-e benne nagybetű

**konfig**

In [143]:
%%writefile HunTag3-master/myconf.yaml

%YAML 1.1
---
default:
  cutoff: 1  #  1 if not set
  radius: 3  # -1 if not set

features:
 - # szóalak
    name: token
    type: token
    actionName: getToken
    fields: 0
 - # morf elemzés
    name: ana
    type: token
    actionName: getAna
    fields: 1
# - # szófajkód
#    name: pos
#    type: 
#    actionName: getPos
#    fields: 
#
# - # lemma
#    name: lem
#    type: 
#    actionName: getLem
#    fields: 
 - # csupa nagybetű-e
    name: isUpper
    type: token
    actionName: isUpper
    fields: 0
# - # nagybetűvel kezdődik-e
#    name: isTitle
#    type: 
#    actionName: isTitle
#    fields: 
# - # van-e benne nagybetű
#    name: hasUpper
#    type: 
#    actionName: hasUpper
#    fields: 
...



Overwriting HunTag3-master/myconf.yaml


**features**

In [144]:
%%writefile HunTag3-master/features.py


def token_getToken(token, _=None):
    return [token]

def token_getAna(ana, _=None):
    return [ana]

#def token_getPos(ana, _=None):
#    return [ana.split('<')[0]]

#def token_getLem(lem, _=None):
#    return [lem]

def token_isUpper(token, _=None):
    return [int(token.isupper())]

#def token_isTitle(token, _=None):
#    return [int(token.istitle())]

#def token_hasUpper(token, _=None):
#    return [int(token.lower() != token)]



Overwriting HunTag3-master/features.py


**megfigyelési modell**

In [145]:
%%bash

cd HunTag3-master/
python3 huntag.py train -i ../huwiki.ner.sample.train.tsv -c myconf.yaml -m NER
ls -l NER.model

-rw-r--r-- 1 mittelholcz mittelholcz 1577355 ápr   24 18:27 NER.model


featurizing sentences...1000...1000...done!
creating training problem...done!
training with option(s) "None"...done
saving model...done
saving feature and label lists...done


**átmenet modell**

In [146]:
%%bash

cd HunTag3-master/
python3 huntag.py transmodel-train -i ../huwiki.ner.sample.train.tsv -m NER

lambda1: 0.07944765574823379
lambda2: 0.7835260115606937
lambda3: 0.1370263326910726


**címkézés**

In [147]:
%%bash

cd HunTag3-master/
python3 huntag.py tag -i ../huwiki.ner.sample.test.tsv -c myconf.yaml -m NER -o huwiki.ner.test.tagged.tsv

loading transition model...done
loading observation model...done
500...done


**kiértékelés**

In [148]:
%%bash

cd HunTag3-master/
python3 utils/eval.py -c -i huwiki.ner.test.tagged.tsv

processed 7232 tokens with 131 phrases;found: 17 phrases; correct: 8.
accuracy:  96.96%; precision:  47.06%;recall:   6.11%; FB1:  10.81
               LOC:precision:   0.00%;recall:   0.00%; FB1:   0.00	0
              MISC:precision:   0.00%;recall:   0.00%; FB1:   0.00	0
               ORG:precision:   0.00%;recall:   0.00%; FB1:   0.00	0
               PER:precision:  47.06%;recall:  26.67%; FB1:  34.04	17
